# Take Home Exam

## Instructions:
* Make sure that you run all code cells that are included in this notebook.
* I will deduct points in **half-point steps** in the following cases:
    * Your code does not work.
    * You are not answering the question (Read carefully).
    * Your code is slow or inefficient.
    * You do not explain why you are doing what you are doing.
    * Clean **unnecessary** elements before submission.
    * **Check the rubric on Camino for this assignment!**
    
**The deadline is Saturday, February 10, 2018, 11:59PM**.

**I am not available on Saturday and have only limited access to my email. If you have questions or issues, make sure you adress them on Friday. **

## Overview

In [82]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.4f}'.format

In [83]:
%matplotlib inline

In [84]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [85]:
cd = pd.read_csv('complications.csv', dtype=str).rename(columns=lambda x: x.strip())

This dataset contains complications in U.S. hospitals. Each observation represents the number of complications and the 30-day death rate for a specific complication in a specific hospital.

The columns are as follows:

In [ ]:
cd.columns

| Column Name | Description|
|-------------|------------|
|Provider ID | Unique ID of the hospital |
|Hospital Name| Name of the hospital|
|Address| Address of the hospital|
|City| Name of the city the hospital is in |
|ZIP Code | Postal code of the hospital|
|County Name | Name of the county the hospital is in|
|Phone Number | Phone number of the hospital |
|Year | The year in which the complications were recorded|
|Measure ID | Code for the complication |
|Measure Name | Name of the complication|
|Denominator| Number of underlying medical procedures|
|Score | Estimate for number of serious complications |
|Lower Estimate | Lower boundary of score|
|Higher Estimate | Upper boundary of score|

A quick overview of the dataset:

In [5]:
cd.describe()

,Provider ID,Hospital Name,Address,City,ZIP Code,County Name,Phone Number,Year,Measure ID,Measure Name,Denominator,Score,Lower Estimate,Higher Estimate
count,272788,272788,272788,272788,272788,272626,272788,272788,272788,272788,272788,272788,272788,272788
unique,4746,5119,4915,2874,4368,1526,5028,6,26,26,23771,6373,8690,11699
top,050291,MEMORIAL HOSPITAL,100 HOSPITAL DRIVE,CHICAGO,76104,LOS ANGELES,6104022273,2013,MORT_30_PN,Pneumonia (PN) 30-Day Mortality Rate,26.0,0.06,0.0,0.06
freq,89,710,378,1957,353,5421,178,58668,24942,24942,1435,2947,23907,2897


A sample of the dataset:

In [6]:
cd.sample(5, random_state=1234).head()

,Provider ID,Hospital Name,Address,City,ZIP Code,County Name,Phone Number,Year,Measure ID,Measure Name,Denominator,Score,Lower Estimate,Higher Estimate
22678,050222,SHARP CHULA VISTA MEDICAL CENTER,751 MEDICAL CENTER COURT,CHULA VISTA,91911,SAN DIEGO,6195025800,2016,MORT_30_PN,Pneumonia (PN) 30-Day Mortality Rate,474.0,14.3,12.1,16.8
22088,050191,ST MARY MEDICAL CENTER,1050 LINDEN AVE,LONG BEACH,90813,LOS ANGELES,5624919000,2012,PSI_14_POSTOP_DEHIS,A wound that splits open after surgery on the...,102.0,1.78,0.0,3.64
111295,201309,MAYO REGIONAL HOSPITAL,897 WEST MAIN STREET,DOVER FOXCROFT,04426,PISCATAQUIS,2075644251,2014,MORT_30_STK,Death rate for stroke patients,43.0,14.1,10.2,18.9
44865,100012,LEE MEMORIAL HOSPITAL,2776 CLEVELAND AVE,FORT MYERS,33901,LEE,2393321111,2012,MORT_30_PN,Pneumonia (PN) 30-Day Mortality Rate,971.0,10.6,8.9,12.5
208974,390194,BLUE MOUNTAIN HOSPITAL-GNADEN HUETTEN CAMPUS,211 NORTH 12TH STREET,LEHIGHTON,18235,CARBON,6073771300,2016,MORT_30_STK,Death rate for stroke patients,72.0,15.7,11.7,20.8


# Transform relevant columns

All columns in the dataset are of type `str` (to preserve the ZIP code information). For **five** columns, str is the wrong type. Change the columns and explain why you changed which column.

### Years are best handled as numbers, if you had computation like gap between years or increase years
### Denominator, Score, Lower Estimate, Higher Estimate all have decimal points and can be involved in computations

In [86]:
# group all the columns that need to be changed
cols= ['Year','Denominator','Score','Lower Estimate','Higher Estimate']
# convert all to numeric
cd[cols]= cd[cols].apply(pd.to_numeric)

In [87]:
# double check the types for all columns
cd.dtypes

Provider ID         object
Hospital Name       object
Address             object
City                object
ZIP Code            object
County Name         object
Phone Number        object
Year                 int64
Measure ID          object
Measure Name        object
Denominator        float64
Score              float64
Lower Estimate     float64
Higher Estimate    float64
dtype: object

# Create a ratio between the score and the denominator

Create a new column in the dataset that contains the ratio value.

In [88]:
# store the calculated percentages in a new column called Ratio
cd['Ratio']= cd.Score/cd.Denominator*100

In [89]:
cd.Ratio.head()

0   1.0393
1   1.0951
2   1.1122
3   1.1829
4   1.1976
Name: Ratio, dtype: float64

# Create an overview of the ratio for deathly complications

0. **Be careful when you identify 'deathly' complications and make sure that you catch all of them.**
1. Create the overview (Make sure that the overview is understandable to everyone.)
2. Choose an appropriate measure to aggregate the data. **Ignore that the data is for multiple years**.
2. Sort with the highest ratio on top.

In [90]:
# explore all complications and find out which are actually deathly 
# my definition of deathly complications is those ones that contain 'deanth' and 'mortality'
cd['Measure Name'].unique()

array(['Rate of complications for hip/knee replacement patients',
       'Acute Myocardial Infarction (AMI) 30-Day Mortality Rate',
       'Death rate for CABG',
       'Death rate for chronic obstructive pulmonary disease (COPD) patients',
       'Heart failure (HF) 30-Day Mortality Rate',
       'Pneumonia (PN) 30-Day Mortality Rate',
       'Death rate for stroke patients',
       'Postoperative Acute Kidney Injury Requiring Dialysis Rate',
       'Postoperative Respiratory Failure Rate',
       'Serious blood clots after surgery',
       'Blood stream infection after surgery',
       'A wound that splits open  after surgery on the abdomen or pelvis',
       'Accidental cuts and tears from medical treatment',
       'Pressure sores',
       'Deaths among Patients with Serious Treatable Complications after Surgery',
       'Collapsed lung due to medical treatment',
       'Infections from a large venous catheter',
       'Broken hip from a fall after surgery',
       'Perioperative H

In [91]:
# create a mask that contains all deathly complications 
Deathly = (cd['Measure Name'].str.lower().str.contains('death')) | (cd['Measure Name'].str.lower().str.contains('mortality'))

In [92]:
# double check I have all deathly complications
cd[Deathly]['Measure Name'].unique()

array(['Acute Myocardial Infarction (AMI) 30-Day Mortality Rate',
       'Death rate for CABG',
       'Death rate for chronic obstructive pulmonary disease (COPD) patients',
       'Heart failure (HF) 30-Day Mortality Rate',
       'Pneumonia (PN) 30-Day Mortality Rate',
       'Death rate for stroke patients',
       'Deaths among Patients with Serious Treatable Complications after Surgery'], dtype=object)

In [94]:
# overview for deathly complications ratios, sorted by the most frequent one 
cd[Deathly].groupby('Measure Name')['Ratio'].describe().sort_values(by='count',ascending=False)

,count,mean,std,min,25%,50%,75%,max
Measure Name,,,,,,,,
Pneumonia (PN) 30-Day Mortality Rate,24942.0000,11.5855,11.6854,0.4417,3.5523,7.1695,15.0467,75.2000
Heart failure (HF) 30-Day Mortality Rate,22275.0000,12.1721,12.0148,0.3340,3.1868,7.1875,17.6190,63.6000
Death rate for chronic obstructive pulmonary disease (COPD) patients,21738.0000,8.3341,7.5125,0.2801,2.7547,5.3637,11.7742,36.4000
Death rate for stroke patients,16593.0000,18.4757,16.1319,0.7004,5.9398,11.7901,27.4000,70.0000
Acute Myocardial Infarction (AMI) 30-Day Mortality Rate,14817.0000,15.9874,14.5168,0.6814,5.2174,10.2000,22.4561,66.8000
Deaths among Patients with Serious Treatable Complications after Surgery,7515.0000,52.4407,23.2875,11.3240,31.5883,49.3492,71.3425,99.9689
Death rate for CABG,3102.0000,4.1510,3.0806,0.1557,1.7742,3.3699,5.6000,17.5000


### The most frequent deathly complication is pneumonia (30-day mortality)
### The least frequent deathly complication is CABG and is 8 times less than that of pneumonia
### The range of the mean of deathly complications is huge from as low as 4% to over 50%
### Most deathly complications have the mean ratios between 10% to 20%
### The standard deviation for each deathly complication varies 

In [15]:
# sorted deathly ratios by mean values
cd[Deathly].groupby('Measure Name')['Ratio'].mean().sort_values(ascending=False)

Measure Name
Deaths among Patients with Serious Treatable Complications after Surgery   52.4407
Death rate for stroke patients                                             18.4757
Acute Myocardial Infarction (AMI) 30-Day Mortality Rate                    15.9874
Heart failure (HF) 30-Day Mortality Rate                                   12.1721
Pneumonia (PN) 30-Day Mortality Rate                                       11.5855
Death rate for chronic obstructive pulmonary disease (COPD) patients        8.3341
Death rate for CABG                                                         4.1510
Name: Ratio, dtype: float64

### The most possible death complication is Deaths among Patients with Serious Treatable Complications after Surgery and is over 50%
### CABG is the least frequent and has the lowest possibility 
### Although Pneumonia is the most frequent one, the mean rate is not that high, 11%, compared to Deaths among Patients with Serious Treatable Complications after Surgery

# Identify the five safest hospitals

Return a **list** with the hospitals that have, on average, the lowest rate of deadly complications. **Be careful when you identify 'deathly' complications.** (Again, ignore that the dataset covers multiple years).

In [100]:
# find the lowest deathly mean ratio by each hospital 
# they are around 2% - 2.5%
temp = cd[Deathly].groupby('Hospital Name')['Ratio'].mean().nsmallest(5)
temp

Hospital Name
UNIVERITY OF MD BALTO WASHINGTON  MEDICAL CENTER   2.0625
NEW YORK-PRESBYTERIAN HOSPITAL                     2.1950
NORTON HOSPITALS, INC                              2.2598
SAN JUAN VA MEDICAL CENTER                         2.3965
FLORIDA HOSPITAL                                   2.5612
Name: Ratio, dtype: float64

In [98]:
# give only the list of hospital names 
list(temp.index)

['UNIVERITY OF MD BALTO WASHINGTON  MEDICAL CENTER',
 'NEW YORK-PRESBYTERIAN HOSPITAL',
 'NORTON HOSPITALS, INC',
 'SAN JUAN VA MEDICAL CENTER',
 'FLORIDA HOSPITAL']

# Augment the datset with information about the state

Read the file `geo-data.csv` into an object `zip_codes`. **Make sure that you read all columns as string.** ([Link](https://github.com/scpike/us-state-county-zip))

In [18]:
# read the file from github and read all columns as strings
zip_codes = pd.read_csv('https://raw.githubusercontent.com/scpike/us-state-county-zip/master/geo-data.csv', dtype=str).rename(columns=lambda x: x.strip())

In [19]:
zip_codes.head()

,state_fips,state,state_abbr,zipcode,county,city
0,1,Alabama,AL,35004,St. Clair,Acmar
1,1,Alabama,AL,35005,Jefferson,Adamsville
2,1,Alabama,AL,35006,Jefferson,Adger
3,1,Alabama,AL,35007,Shelby,Keystone
4,1,Alabama,AL,35010,Tallapoosa,New site


This file contains information about the relationship of zip codes, counties, and U.S. states. 

In [23]:
zip_codes.columns

Index(['state_fips', 'state', 'state_abbr', 'zipcode', 'county', 'city'], dtype='object')

| Column Name | Description|
|-------------|------------|
|state_fips | FIPS code for the state |
|state| Name of the state|
|state_abbr| Abbreviation of the state|
|zipcode | Postal code|
|county | County name|
|city | City name  |

1. Combine the dataset on complications with the dateset on zip codes.
2. Explain why you chose the column that used to combine the datasets.
3. **Do not worry, not all states will be matched.**

In [101]:
# keep only one zip code column 
zip_codes = zip_codes.rename(columns={'zipcode':'ZIP Code'})

In [102]:
# left join two datasets on zip codes, because zip code is the most accurate one 
# (others like city and county, people might spell them wrong)
new = pd.merge(cd, zip_codes, on ='ZIP Code',  how='left')

In [26]:
# double check all the columns are included 
new.columns

Index(['Provider ID', 'Hospital Name', 'Address', 'City', 'ZIP Code',
       'County Name', 'Phone Number', 'Year', 'Measure ID', 'Measure Name',
       'Denominator', 'Score', 'Lower Estimate', 'Higher Estimate', 'Ratio',
       'state_fips', 'state', 'state_abbr', 'county', 'city'],
      dtype='object')

# Create an overview of the complications per U.S. state

* Return a sorted list with the overall complication rate per state. 
* Ignore that the data is for multiple years. 
* Focus on the **best** ten states.

In [103]:
# find out the 10 states with the lowest complication mean rates
# they are all below 10% 
# no western states, they are all east coast state plus one midwest state Ohio
new.groupby('state')['Ratio'].mean().nsmallest(10)

state
Delaware               6.3448
Maryland               6.8499
Florida                7.5640
District of columbia   7.7743
Virginia               7.9625
Pennsylvania           8.6943
New york               8.8056
South carolina         8.9254
North carolina         8.9914
Ohio                   9.1787
Name: Ratio, dtype: float64

In [31]:
# overview of the complications rate per U.S. state 
# not all states are presented here
# the best 10 states are all in the top 25% quantile 
new.groupby('state')[['Ratio']].mean().describe()

,Ratio
count,44.0000
mean,11.9016
std,3.4335
min,6.3448
25%,9.3718
50%,10.7443
75%,13.9576
max,21.0053


# Create an overview of the change in complications across U.S. states

* Focus on the deadly complications only.
* Calculate the change as follows: **(average rate for last year in dataset - average rate for first year in dataset)/(average rate for first year in dataset)**

In [106]:
# find out the first and the last years in the dataset
cd.Year.agg(['min','max'])

min    2011
max    2016
Name: Year, dtype: int64

In [107]:
# recreate the deathly complication mask for the combined dataset
Deathly = (new['Measure Name'].str.lower().str.contains('death')) | (new['Measure Name'].str.lower().str.contains('mortality'))

In [108]:
# calculate the mean deathly ratio for the last year by states
lastyear= new[Deathly][new['Year'] == 2016].groupby('state')['Ratio'].mean()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [109]:
# calculate the mean deathly ratio for the first year by states
firstyear= new[Deathly][new['Year'] == 2011].groupby('state')['Ratio'].mean()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [110]:
# calculate the change
Change = (lastyear-firstyear)/firstyear

In [115]:
Change.nsmallest(10)

state
Delaware         -0.1874
South carolina   -0.1660
Nevada           -0.1612
Arizona          -0.1380
Oregon           -0.1280
Maryland         -0.1023
Alaska           -0.0912
Tennessee        -0.0843
Alabama          -0.0736
Virginia         -0.0683
Name: Ratio, dtype: float64

In [59]:
# overview of the change across states
# a negative mean of the change tells an overall improvement on the deathly ratio across states
# the absolute difference between the max and min is quite large, more than 8%
# the best 10 states are all in the top 25% quantile 
Change.describe()

count   44.0000
mean    -0.0282
std      0.0689
min     -0.1874
25%     -0.0622
50%     -0.0264
75%      0.0106
max      0.1061
Name: Ratio, dtype: float64

Which state has improved the most?

In [62]:
# Delaware has a nearly 20% decrease of deathly ratio from the first year to the last year
Change.nsmallest(1)

state
Delaware   -0.1874
Name: Ratio, dtype: float64

Which state has the worst development?

In [63]:
# Wyoming has a little over 10% increase of deathly ratio from the first year to the last year
# Wyoming goverment should have some investigation on how to improve the deathly rate 
Change.nlargest(1)

state
Wyoming   0.1061
Name: Ratio, dtype: float64

### Call to action: even though on average there is an improvement on the deathly rate across these states, the number is as low as 2.82%. Those staes in the top 75% quantile should have paid more attention on how to lower the deathly rate while try not to exaccerbate it even more

# Bonus

Which state has the worst development for 'Deaths among Patients with Serious Treatable Complications after Surgery'?

In [66]:
# take the slice form the dateset for easy calculation later
STC= new[new['Measure Name']=='Deaths among Patients with Serious Treatable Complications after Surgery']

In [68]:
# find out the year range of the slice 
STC.Year.unique()

array([2012, 2013, 2014, 2015])

In [69]:
# calculate the mean deathly ratio for the last year
last15 = STC[STC.Year==2015].groupby('state')['Ratio'].mean()

In [81]:
# calculate the mean deathly ratio for the first year
first12 = STC[STC.Year==2012].groupby('state')['Ratio'].mean()

In [71]:
# calculate the change
STCchange=(last15 - first12)/first12

In [72]:
# the state with the max positive value of change has the worst development for this measure
STCchange.nlargest(1)

state
Utah   0.2155
Name: Ratio, dtype: float64